# Pro-tips- the challenge here is that with the netcdf in its origional configuration it was not possible to read all the data in to downsample it. So what we did was to read in each file, swap the time and obs dimensions and then only keep the 
1. Create a directory in you data files for downsampled .ncdfs to live

In [2]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import re
import requests
import pickle as pk
import os
import gc
#!pip install netcdf4 == 1.5.0

In [7]:
# make the output directory
new_dir = '/home/jovyan/data/botpt/RS03CCAL-MJ03F-05-BOTPTA301/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [8]:
# Sensor E url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T034014801Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T174216826Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191026T023346674Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
# Sensor D:url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/catalog.html'

In [9]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i + '#fillmismatch') for i in x]
#datasets

## origional loop

In [ ]:
# num = 0
# for i in datasets:
#     ds = xr.open_dataset(i)
#     ds = ds.swap_dims({'obs': 'time'})

#     pressure_min = pd.DataFrame()
#     pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
#     del pressure_min.index.name

#     pressure_min = pressure_min.dropna()

#     out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.pd'
#     num = num +1

#     with open(out, 'wb') as fh:
#         pk.dump(pressure_min,fh)

#     gc.collect()

# pd routines

In [10]:
%%time
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = new_dir + i.split('/')[-1][:-16] + '_resampled' + '.nc'
    ds = xr.Dataset.from_dataframe(pressure_min)
    ds.to_netcdf(out)

CPU times: user 14min 8s, sys: 6min 45s, total: 20min 54s
Wall time: 41min 20s


In [27]:
i=datasets[1]
ds = xr.open_dataset(i)
ds = ds.swap_dims({'obs': 'time'})

pressure_min = pd.DataFrame()
pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
del pressure_min.index.name

pressure_min = pressure_min.dropna()

out = new_dir + i.split('/')[-1][:-16]
out

'/home/jovyan/data/botpt/RS03ECAL-MJ03E-06-BOTPTA302/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20191004T120000-20191012T055959.950000'

## Full NetCDF download

In [ ]:
%%time
import subprocess
download_data = 'wget -r -l1 -nd -nc -np -e robots=off -A.nc --no-check-certificate https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T034014801Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/'
subprocess.call(download_data, shell=True) 

# Test Loop PD

In [ ]:
# %%time
# i=datasets[1]
# ds = xr.open_dataset(i)
# ds = ds.swap_dims({'obs': 'time'})

# pressure_min = pd.DataFrame()
# pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
# del pressure_min.index.name

# pressure_min = pressure_min.dropna()
# out = 'minute_mean_data/test.nc'

# ds = xr.Dataset.from_dataframe(pressure_min)
# ds.to_netcdf(out)

## xr routine 

In [ ]:
%%time
for i in datasets:
    ds = xr.open_mfdataset([i], parallel =True, combine='by_coords')
    ds = ds.swap_dims({'obs': 'time'})
    ds= ds['bottom_pressure'].to_dataset()
    ds=ds.resample(time='T').mean()
    out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled_test' + '.nc'
    ds.to_netcdf(out)

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.128.212:46185")
client

In [ ]:
%%time
for i in datasets:
    ds = xr.open_mfdataset([i], parallel =True, combine='by_coords'
    out = 'new_dir' + i.split('/')[-1][:-3] + 'dask'
    ds.to_netcdf(out)

# Least Efficienct

In [ ]:
# %%time
# i= datasets[1]
# ds = xr.open_mfdataset([i], parallel=True, drop_variables= ['deployment', 'id', 'bottom_pressure_qc_executed', 'date_time_string'
#                                                   , 'driver_timestamp', 'ingestion_timestamp', 'internal_timestamp'
#                                                   ,'port_timestamp', 'preferred_timestamp', 'press_trans_temp'
#                                                   ,'provenance','sensor_id', 'time_sync_flag', 'bottom_pressure_qc_results']
#                             )
# ds=ds.swap_dims({'obs': 'time'}).resample(time='T').mean()
# out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.nc'
# ds.to_netcdf(out)